In [ ]:
import SAGA
from easyquery import Query, QueryMaker
from SAGA.database import FitsTable
from SAGA import ObjectCuts as C
from SAGA import HostCuts as H

from itertools import combinations
from collections import Counter, defaultdict
from astropy.table import Table, join, vstack
from astropy.coordinates import SkyCoord, Distance
from SAGA.utils import add_skycoord
from easyquery import Query, QueryMaker
from astropy.io import ascii

import SAGA
from SAGA.database import FitsTable, GoogleSheets
from SAGA import ObjectCuts as C
from SAGA.utils import add_skycoord, fill_values_by_query
from SAGA.utils.distance import z2v, d2m, m2d
from SAGA.utils.display import show_images
from SAGA.objects.object_catalog import calc_fiducial_p_sat, calc_fiducial_p_sat_corrected
from scipy.stats import spearmanr
print(SAGA.__version__)

In [ ]:
saga = SAGA.QuickStart()
saga.set_default_base_version("paper2")

In [ ]:
masterlist = saga.host_catalog.load_master_list()
hostlist = saga.host_catalog.load(include_stats=True)

In [ ]:
def annotate_base(base):
    base["Mr_"] = base["r_mag"] - d2m(base["HOST_DIST"])
    base["Mr"] = np.where(np.isfinite(base["Mr"]) & C.has_spec.mask(base), base["Mr"], base["Mr_"])
    base["log_sm"] = np.where(np.isfinite(base["log_sm"]), base["log_sm"], 1.254 + 1.0976 * base["gr"] - 0.4 * base["Mr"])
    
    base["human_selected"] = 0
    hs = saga.database["human_selected"].read()
    for i in range(1, 4):
        base["human_selected"] += np.in1d(base["OBJID"], Query(f"score == {i}").filter(hs, "OBJID")).astype(np.int) * i

    return base

In [ ]:
base = SAGA.database.FitsTable("/home/yymao/Downloads/saga_base_all.fits").read()
base = saga.host_catalog.construct_host_query("paper2_complete").filter(base)
base = annotate_base(base)

In [ ]:
print(C.is_sat.count(base), Query(C.is_sat, C.r_abs_limit).count(base))

In [ ]:
print(
    Query(C.is_sat, ~C.paper2_targeting_cut).count(base), 
    Query("RHOST_KPC > 10", C.has_spec, "SPEC_Z > 0.002", "SPEC_Z < 0.015", ~C.paper2_targeting_cut).count(base),
)

In [ ]:
print(C.is_sat.count(base), len(set(Query(C.is_sat).filter(base, "OBJID"))))

 The inner distance and velocity limits exclude 96 hosts in fields

In [ ]:
nearbr_host_criteria = [
    "abs(GLAT) >= 25",
    "BRIGHTEST_STAR_R1 >= 5",
    "M_HALO < 13",
    QueryMaker.equals("REMOVED_BY_HAND", "False"),
    "BRIGHTEST_K_R1 >= K_TC + 1.6",
    "K_ABS >= -24.6",
    "K_ABS <= -23.0",
]

has_ls = Query("COVERAGE_DECALS_DR6 >= 0.99") | "COVERAGE_DECALS_DR7 >= 0.99"
has_des = Query("COVERAGE_DES_DR1 >= 0.99")
has_sdss = Query("COVERAGE_SDSS >= 0.99")
has_any = (has_ls | has_des | has_sdss)

nearbr_hosts = Query(Query("DIST < 25") | "V_HELIO < 1400", has_any, *nearbr_host_criteria).filter(masterlist)
print(len(nearbr_hosts))

In [ ]:
lv_hosts = ["NGC1023", "NGC1156", "NGC2903", "NGC4258", "NGC4565", "NGC4631", "NGC5023", "NGC5194", "NGC4826", "NGC4594", "NGC4736"]
lv_hosts1 = QueryMaker.in1d("COMMON_NAME", lv_hosts).filter(masterlist)
assert len(lv_hosts1) == len(lv_hosts)

for q in nearbr_host_criteria:
    lv_hosts1[str(q)] = Query(q).mask(lv_hosts1)

In [ ]:
Query(nearbr_host_criteria[2], nearbr_host_criteria[4], *nearbr_host_criteria[-2:]).filter(lv_hosts1, "COMMON_NAME")

In [ ]:
lv_hosts1

 \some{}\% of galaxies identified in the range $0.015 < z < 0.03$ are outside of the primary targeting region,

In [ ]:
Query(C.basic_cut2, "SPEC_Z < 0.015", "SPEC_Z > 0.002", C.has_spec, ~C.paper2_targeting_cut, "SATS != 3").count(base)

In [ ]:
Query(C.basic_cut2, "SPEC_Z >= 0.015", "SPEC_Z < 0.03", C.has_spec, ~C.paper2_targeting_cut).count(base) / Query(C.basic_cut2, "SPEC_Z >= 0.015", "SPEC_Z < 0.03", C.has_spec).count(base)

Among \nhoststotal{} hosts, \nhosts{} have spectroscopic coverage higher than 80\% (and 22 have coverage higher than 90\%). An additional 19 hosts have at least some SAGA redshifts.

In [ ]:
Query("paper2_need_spec / paper2_total < 0.1", H.good).count(hostlist), Query("paper2_need_spec / paper2_total < 0.2", H.good).count(hostlist)

In [ ]:
Query("paper2_need_spec / paper2_total > 0.2", "specs_ours_rvir > 50", H.good).count(hostlist)

 where we find \nsats{} satellites among 5,418 redshifts. Outside of the primary targeting region, we find no satellites among 21,310 redshifts.

In [ ]:
Query(C.basic_cut2, C.has_spec, C.paper2_targeting_cut).count(base)

In [ ]:
Query(C.basic_cut2, C.has_spec, ~C.paper2_targeting_cut).count(base)

we visually inspected targets in SAGA field to identify all dLSBGs and found \checknumber{70} of them that we have not obtained redshifts (down to $M_{r,o} < -12.3$), and \checknumber{19}\% of them are slightly outside the primary targeting region.

In [ ]:
Query(C.basic_cut2, ~C.has_spec, "Mr_ < -12.295", "human_selected > 0").count(base), Query(C.basic_cut2, ~C.has_spec, "Mr_ < -12.295", "human_selected > 0", ~C.paper2_targeting_cut).count(base)

In [ ]:
Query(C.is_sat, C.r_abs_limit, "EW_Halpha < 2").count(base)

Specifically, the relative fraction of SAGA to literature redshifts is about 42\% for bright galaxies ($r_o < 19$) and 81\% for faint galaxies ($r_o > 19$). 

In [ ]:
Query(C.has_spec, C.basic_cut2, C.has_our_specs_only, "r_mag > 19").count(base) / Query(C.has_spec, C.basic_cut2, "r_mag > 19").count(base)

In [ ]:
Query(C.has_spec, C.basic_cut2, C.has_our_specs_only, "r_mag < 19").count(base) / Query(C.has_spec, C.basic_cut2, "r_mag < 19").count(base)

In [ ]:
Query(C.has_spec, C.basic_cut2, C.has_our_specs_only, "r_mag > 17.5").count(base) / Query(C.has_spec, C.basic_cut2, "r_mag > 17.5").count(base)

the number of targets that are within the primary targeting region and between 10--300 kpc to each host in projection is about \checknumber{200} ($\sim$\,200 targets per squared degree).

In [ ]:
Query(C.basic_cut2, "RHOST_KPC > 10", C.paper2_targeting_cut).count(base) / 36

In [ ]:
Query(C.basic_cut2, "RHOST_KPC > 10", C.paper2_targeting_cut).count(base) / 29.1787365

Among the fainter half of the satellites (54 satellites with $-15.7 < M_{r,o} < -12.3$), $47/54$ (87\%) have the first redshift measurement from SAGA. 

In [ ]:
Query(C.is_sat, C.r_abs_limit,  C.has_our_specs_only).count(base) / Query(C.is_sat, C.r_abs_limit).count(base)

In [ ]:
np.median(Query(C.is_sat, C.r_abs_limit).filter(base, "Mr"))

In [ ]:
Query(C.is_sat, C.r_abs_limit, "Mr >= -15.77", C.has_our_specs_only).count(base), Query(C.is_sat, C.r_abs_limit, "Mr >= -15.77").count(base)

In [ ]:
Query(C.basic_cut2, C.has_our_specs_only, ~saga.host_catalog.construct_host_query("paper1_complete")).count(base) / 28

LMC-like sats

In [ ]:
Counter(Query(C.is_sat, "Mr < -18").filter(base,"HOSTID"))

In [ ]:
Counter(Query(C.is_sat, "log_sm > 9").filter(base,"HOSTID"))

In [ ]:
spearmanr(*np.array([(t["Mr"].min(), C.r_abs_limit.count(t)) for t in C.is_sat.filter(base, ["Mr", "HOSTID"]).group_by(["HOSTID"]).groups]).T)

In [ ]:
spearmanr(*np.array([(t["Mr"].min(), t["HOST_MK"][0]) for t in C.is_sat.filter(base, ["Mr", "HOSTID", "HOST_MK"]).group_by(["HOSTID"]).groups]).T)

In [ ]:
chosts = H.paper2_complete.filter(hostlist)
print(len(chosts))
spearmanr(chosts["K_ABS"], chosts["sats_Mr_limit"])

 These nearby hosts include the well-studied Milky Way analogs M94 \citep{Smercina2018} and M101 \citep{Bennet2019a}, as well as 2 of the 10 hosts (NGC\,2903 and NGC\,4258) presented in \citet{Carlsten2019}. 